### 라이브러리 불러오기

In [89]:
# !pip install timm
import warnings
warnings.filterwarnings('ignore')
from copy import deepcopy
from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random


import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time
import torch.cuda

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0) # gpu 1개 이상일 때 

# 넘파이 랜덤시드 고정
np.random.seed(0)

# 파이썬 랜덤시드 고정
random.seed(0)

In [90]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"  # Set the GPU 2 to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [91]:
train_df = pd.read_csv("train_df_add_data.csv")

In [92]:
submission= pd.DataFrame(columns = ["class","state"])

In [93]:
label_split = train_df['label'].str.split("-")
submission["class"] = label_split.str.get(0)
submission["state"] = label_split.str.get(1)

In [94]:
train_df = pd.concat([train_df,submission],axis=1)

In [95]:
train_df

,Unnamed: 0,file_name,label,class,state
0,0,30.png,transistor-bent_lead,transistor,bent_lead
1,1,31.png,transistor-damaged_case,transistor,damaged_case
2,2,32.png,transistor-misplaced,transistor,misplaced
3,3,33.png,transistor-misplaced,transistor,misplaced
4,4,34.png,transistor-cut_lead,transistor,cut_lead
...,...,...,...,...,...
10611,6131,23131.png,wood-liquid,wood,liquid
10612,6132,23132.png,wood-liquid,wood,liquid
10613,6133,23133.png,wood-liquid,wood,liquid
10614,6134,23134.png,wood-liquid,wood,liquid


print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

# 데이터 인코딩

In [96]:
train_class = train_df["class"]

class_unique = sorted(np.unique(train_class))
class_unique_enc =  {key:value for key,value in zip(class_unique, range(len(class_unique)))}

In [97]:
def make_unique(c_name):
    class_df =  train_df[train_df['class']==c_name]
    state_df = class_df['state']
    state_unique = sorted(np.unique(state_df))
    state_unique =  {key:value for key,value in zip(state_unique, range(len(state_unique)))}
    return state_unique

In [98]:
all_state_unique = []
for i in class_unique:
    all_state_unique.append(make_unique(str(i)))


In [99]:
print("클래스 : \n",class_unique_enc)
print("상태값 : \n",all_state_unique[1])

클래스 : 
 {'bottle': 0, 'cable': 1, 'capsule': 2, 'carpet': 3, 'grid': 4, 'hazelnut': 5, 'leather': 6, 'metal_nut': 7, 'pill': 8, 'screw': 9, 'tile': 10, 'toothbrush': 11, 'transistor': 12, 'wood': 13, 'zipper': 14}
상태값 : 
 {'bent_wire': 0, 'cable_swap': 1, 'combined': 2, 'cut_inner_insulation': 3, 'cut_outer_insulation': 4, 'good': 5, 'missing_cable': 6, 'missing_wire': 7, 'poke_insulation': 8}


In [100]:
class_label = [class_unique_enc[k] for k in train_class]
state_df = train_df[["file_name","class",'state']]
state_df_enc = []
all_state_label=[]
for c,i in enumerate(class_unique):
    class_df =  state_df[state_df['class']==str(i)].reset_index(drop=True)
    class_state = class_df["state"]
    state_label = [all_state_unique[c][k] for k in class_state]
    encdf = pd.DataFrame(state_label,columns = ['encode'])
    le_df = pd.concat([class_df,encdf],axis=1)

    state_df_enc.append(le_df)

클래스 라벨 : class_label <br>
각 클래스별 상태 데이터 프레임 : state_df_enc [0~14]   <br>

0 bottle<br>
1 cable<br>
2 capsule<br>
3 carpet<br>
4 grid<br>
5 hazelnut<br>
6 leather<br>
7 metal_nut<br>
8 pill<br>
9 screw<br>
10 tile<br>
11 toothbrush<br>
12 transistor<br>
13 wood<br>
14 zipper

In [101]:
test_df = pd.read_excel("답지.xlsx")
test_labels = test_df["class"]
test_labels = [class_unique_enc[k] for k in test_labels]

In [102]:
state_df_enc

[     file_name   class          state  encode
 0     3740.png  bottle  contamination       2
 1     3741.png  bottle   broken_large       0
 2     3742.png  bottle   broken_small       1
 3     3743.png  bottle   broken_small       1
 4     3744.png  bottle   broken_large       0
 ..         ...     ...            ...     ...
 615  22215.png  bottle   broken_small       1
 616  22216.png  bottle   broken_small       1
 617  22217.png  bottle   broken_small       1
 618  22218.png  bottle   broken_small       1
 619  22219.png  bottle   broken_small       1
 
 [620 rows x 4 columns],
      file_name  class                 state  encode
 0    31690.png  cable             bent_wire       0
 1    31691.png  cable         missing_cable       6
 2    31692.png  cable  cut_outer_insulation       4
 3    31693.png  cable  cut_inner_insulation       3
 4    31694.png  cable  cut_outer_insulation       4
 ..         ...    ...                   ...     ...
 754  23071.png  cable       poke_insu

# 모델

### 클래스 모델 데이터 생성

In [103]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (512, 512))
    return img

In [104]:
train_png = train_df["file_name"].tolist()
test_png = test_df ["file_name"].tolist()

In [105]:
train_path =[]
for i in train_png:
    train_path.append("all_data\\"+i)
train_path= train_path[:]

#test
test_path =[]
for i in test_png:
    test_path.append("test\\"+i)
test_path= test_path[:]

In [106]:
train_imgs = [img_load(m) for m in tqdm(train_path[:])]
test_imgs = [img_load(n) for n in tqdm(test_path[:])]

100%|██████████| 1000/1000 [00:17<00:00, 55.95it/s]


### 클래스 모델구성

In [107]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        img = transforms.ToTensor()(img)
        if self.mode=='test':
            pass
        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=False, num_classes=15)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [108]:
batch_size = 128
epochs = 20

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(class_label), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = Custom_dataset(np.array(test_imgs), np.array(test_labels), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [109]:
class FocalLoss(nn.Module):

    def __init__(self, gamma=5.0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = nn.CrossEntropyLoss(reduction="none")

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

### 클래스 모델 학습

In [110]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

_model = Network().cuda()
model = nn.DataParallel(_model).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = FocalLoss()
# criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 
best_f1 =0
best_f1_model = None
best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    
    model.eval()
    f_pred = [] 
    test_pred=[]
    test_y = []
    with torch.no_grad():
        for batch in (test_loader):
            x = torch.tensor(batch[0], dtype = torch.float32, device = device)
            y = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                f_pred = model(x)
            test_pred.extend(f_pred.argmax(1).detach().cpu().numpy().tolist())
            test_y += y.detach().cpu().numpy().tolist()
    test_f1 = score_function(test_y, test_pred)
    
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'Test     f1 : {test_f1:.5f}')
    if train_f1 > best_f1:
          best_f1 = train_f1
          best_f1_model = deepcopy(model.state_dict())
          print("save best pt")
torch.cuda.empty_cache()

epoch : 1/20    time : 67s/1268s
TRAIN    loss : 0.17100    f1 : 0.87911
Test     f1 : 0.94785
save best pt
epoch : 2/20    time : 66s/1193s
TRAIN    loss : 0.03514    f1 : 0.94364
Test     f1 : 0.93171
save best pt
epoch : 3/20    time : 67s/1132s
TRAIN    loss : 0.03866    f1 : 0.97749
Test     f1 : 0.74781
save best pt
epoch : 4/20    time : 66s/1062s
TRAIN    loss : 0.00992    f1 : 0.97793
Test     f1 : 0.98399
save best pt
epoch : 5/20    time : 66s/996s
TRAIN    loss : 0.00036    f1 : 0.99818
Test     f1 : 0.98396
save best pt
epoch : 6/20    time : 67s/936s
TRAIN    loss : 0.00015    f1 : 0.99947
Test     f1 : 0.98539
save best pt
epoch : 7/20    time : 66s/863s
TRAIN    loss : 0.01974    f1 : 0.99795
Test     f1 : 0.97831
epoch : 8/20    time : 67s/807s
TRAIN    loss : 0.02820    f1 : 0.96999
Test     f1 : 0.98542
epoch : 9/20    time : 67s/736s
TRAIN    loss : 0.02968    f1 : 0.98911
Test     f1 : 0.90595
epoch : 10/20    time : 67s/671s
TRAIN    loss : 0.02701    f1 : 0.99049

KeyboardInterrupt: 

### 클래스 추론

In [111]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())
        
# label_decoder = {val:key for key, val in class_unique_enc.items()}
# f_result = [label_decoder[result] for result in f_pred]
f_result = f_pred

cpd = pd.DataFrame(f_result,columns = ['class'])
class_pred_df = pd.concat([test_df["file_name"],cpd],axis=1)

In [112]:
torch.cuda.empty_cache()

### 스테이트 모델 데이터 생성

In [113]:
def load_state_data(class_num):
    train_path=[]
    train_png = state_df_enc[class_num]["file_name"].tolist()
    for i in train_png:
        train_path.append("all_data\\"+i)
    lstate_imgs = [img_load(m) for m in tqdm(train_path[:])]
    
    lstate_label = state_df_enc[class_num]["encode"].tolist()
    
    return lstate_imgs , lstate_label,len(set(lstate_label))

### 스테이트 모델 구성

In [114]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train',transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
        self.transforms = transforms
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        img = self.transforms(img)
        if self.mode=='test':
            pass
        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,num_classes):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b5', pretrained=False, num_classes=num_classes)
    def forward(self, x):
        x = self.model(x)
        return x

In [115]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(180, expand=True),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),  
    transforms.Resize((512, 512)),  
    transforms.ToTensor(), 
    transforms.Normalize([0.43303847,0.4034577, 0.39415097], [0.18344551,0.17549995, 0.1647388])])
    
    
# train_transform = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.RandomRotation(180, expand=True),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),  
#     transforms.Resize((512, 512)),  
#     transforms.ToTensor(), 
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225]),
#             ])

In [116]:
def train_start(class_num, epochs, count):
    s_model = Network(class_num).cuda()
    smodel = nn.DataParallel(s_model).to(device)

    optimizer = torch.optim.Adam(smodel.parameters(), lr=1e-3)
    criterion = FocalLoss()
    criterion = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler() 
    best_f1 =0
    best_f1_model = None
    best=0
    
#     tem_df = test_df[test_df['class']==count]
#     tem_png = tem_df["file_name"].tolist()
#     tem_path =[]
#     for i in tem_png:
#         tem_path.append("test\\"+i)
#     tem_imgs = [img_load(m) for m in tqdm(tem_path[:])]
#     answer_state = test_df['state']
#     print(all_state_unique[0])
#     answer_state = [all_state_unique[0][k] for k in answer_state]
#     test_state_dataset = Custom_dataset(np.array(tem_imgs), np.array(answer_state), mode='test', transforms=train_transform)
#     test_state_loader = DataLoader(test_state_dataset, shuffle=False, batch_size=batch_size)
                    
    for epoch in range(epochs):
        start=time.time()
        train_loss = 0
        train_pred=[]
        train_y=[]
        smodel.train()
        for batch in (train_loader):
            optimizer.zero_grad()
            x = torch.tensor(batch[0], dtype=torch.float32, device=device)
            y = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                pred = smodel(x)
            loss = criterion(pred, y)


            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        
            train_loss += loss.item()/len(train_loader)
            train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
            train_y += y.detach().cpu().numpy().tolist()
    
#         smodel.eval()
#         f_pred = [] 
#         test_pred=[]
#         test_y = []
#         with torch.no_grad():
#             for batch in (test_state_loader):
#                 x = torch.tensor(batch[0], dtype = torch.float32, device = device)
#                 y = torch.tensor(batch[1], dtype=torch.long, device=device)
#                 with torch.cuda.amp.autocast():
#                     f_pred = smodel(x)
#                 test_pred.extend(f_pred.argmax(1).detach().cpu().numpy().tolist())
#                 test_y += y.detach().cpu().numpy().tolist()
#         test_f1 = score_function(test_y, test_pred)
     
    
        train_f1 = score_function(train_y, train_pred)

        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
#         print(f'Test     f1 : {test_f1:.5f}')
        if train_f1 > best_f1:
              best_f1 = train_f1
              best_f1_model = deepcopy(smodel.state_dict())
              print("save best pt")
    torch.cuda.empty_cache()
#     torch.save(smodel, str(class_num) + 'model.pt') 
    return smodel

In [117]:
state_models = []
batch_size = 32
epochs = 40

for count in range(15):

    state_imgs , state_label,num_classes = load_state_data(count)

    train_dataset = Custom_dataset(np.array(state_imgs), np.array(state_label), mode='train',transforms=train_transform)
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    
    
    smodel = train_start(num_classes,epochs,count)
    state_models.append(smodel)
    print(count)



100%|██████████| 620/620 [00:08<00:00, 72.44it/s]


epoch : 1/40    time : 14s/555s
TRAIN    loss : 8.16133    f1 : 0.26368
save best pt
epoch : 2/40    time : 14s/530s
TRAIN    loss : 2.37061    f1 : 0.25754
epoch : 3/40    time : 14s/514s
TRAIN    loss : 1.47256    f1 : 0.28503
save best pt
epoch : 4/40    time : 14s/505s
TRAIN    loss : 1.45088    f1 : 0.28439
epoch : 5/40    time : 14s/486s
TRAIN    loss : 1.31882    f1 : 0.36128
save best pt
epoch : 6/40    time : 14s/471s
TRAIN    loss : 1.13589    f1 : 0.46095
save best pt
epoch : 7/40    time : 14s/462s
TRAIN    loss : 1.12307    f1 : 0.49223
save best pt
epoch : 8/40    time : 14s/447s
TRAIN    loss : 1.04983    f1 : 0.54550
save best pt
epoch : 9/40    time : 14s/443s
TRAIN    loss : 1.02278    f1 : 0.52399
epoch : 10/40    time : 14s/430s
TRAIN    loss : 0.87219    f1 : 0.61978
save best pt
epoch : 11/40    time : 14s/418s
TRAIN    loss : 0.87874    f1 : 0.62447
save best pt
epoch : 12/40    time : 14s/390s
TRAIN    loss : 0.85320    f1 : 0.66659
save best pt
epoch : 13/40   

100%|██████████| 759/759 [00:14<00:00, 53.40it/s]


epoch : 1/40    time : 17s/677s
TRAIN    loss : 4.37410    f1 : 0.14397
save best pt
epoch : 2/40    time : 17s/649s
TRAIN    loss : 2.21842    f1 : 0.25800
save best pt
epoch : 3/40    time : 17s/621s
TRAIN    loss : 1.72778    f1 : 0.31673
save best pt
epoch : 4/40    time : 17s/610s
TRAIN    loss : 1.62341    f1 : 0.36710
save best pt
epoch : 5/40    time : 17s/593s
TRAIN    loss : 1.40287    f1 : 0.44787
save best pt
epoch : 6/40    time : 17s/580s
TRAIN    loss : 1.29950    f1 : 0.46675
save best pt
epoch : 7/40    time : 17s/560s
TRAIN    loss : 1.09802    f1 : 0.56177
save best pt
epoch : 8/40    time : 17s/544s
TRAIN    loss : 0.98108    f1 : 0.66468
save best pt
epoch : 9/40    time : 17s/525s
TRAIN    loss : 0.87297    f1 : 0.67015
save best pt
epoch : 10/40    time : 17s/507s
TRAIN    loss : 0.79831    f1 : 0.70176
save best pt
epoch : 11/40    time : 17s/490s
TRAIN    loss : 0.66665    f1 : 0.73966
save best pt
epoch : 12/40    time : 17s/481s
TRAIN    loss : 0.62682    f1 

100%|██████████| 876/876 [00:15<00:00, 55.19it/s]


...

epoch : 1/40    time : 9s/363s
TRAIN    loss : 7.41346    f1 : 0.19126
save best pt
epoch : 2/40    time : 9s/337s
TRAIN    loss : 3.87861    f1 : 0.27642
save best pt
epoch : 3/40    time : 9s/325s
TRAIN    loss : 1.75541    f1 : 0.29879
save best pt
epoch : 4/40    time : 9s/322s
TRAIN    loss : 1.42308    f1 : 0.29557
epoch : 5/40    time : 9s/306s
TRAIN    loss : 1.20620    f1 : 0.39148
save best pt
epoch : 6/40    time : 9s/303s
TRAIN    loss : 1.18243    f1 : 0.40440
save best pt
epoch : 7/40    time : 9s/296s
TRAIN    loss : 1.04011    f1 : 0.44461
save best pt
epoch : 8/40    time : 9s/284s
TRAIN    loss : 0.99899    f1 : 0.59168
save best pt
epoch : 9/40    time : 9s/272s
TRAIN    loss : 0.94302    f1 : 0.55258
epoch : 10/40    time : 9s/268s
TRAIN    loss : 0.80292    f1 : 0.57834
epoch : 11/40    time : 9s/259s
TRAIN    loss : 0.84634    f1 : 0.59783
save best pt
epoch : 12/40    time : 9s/253s
TRAIN    loss : 0.96792    f1 : 0.57169
epoch : 13/40    time : 9s/239s
TRAIN    

100%|██████████| 553/553 [00:10<00:00, 50.33it/s]


epoch : 1/40    time : 13s/517s
TRAIN    loss : 5.55230    f1 : 0.15941
save best pt
epoch : 2/40    time : 13s/476s
TRAIN    loss : 2.12185    f1 : 0.18478
save best pt
epoch : 3/40    time : 12s/462s
TRAIN    loss : 1.68880    f1 : 0.15940
epoch : 4/40    time : 13s/451s
TRAIN    loss : 1.62543    f1 : 0.19575
save best pt
epoch : 5/40    time : 12s/432s
TRAIN    loss : 1.52040    f1 : 0.26064
save best pt
epoch : 6/40    time : 12s/423s
TRAIN    loss : 1.49799    f1 : 0.21893
epoch : 7/40    time : 13s/420s
TRAIN    loss : 1.46229    f1 : 0.28831
save best pt
epoch : 8/40    time : 13s/410s
TRAIN    loss : 1.43864    f1 : 0.30335
save best pt
epoch : 9/40    time : 13s/398s
TRAIN    loss : 1.34223    f1 : 0.31300
save best pt
epoch : 10/40    time : 13s/380s
TRAIN    loss : 1.31302    f1 : 0.34644
save best pt
epoch : 11/40    time : 13s/365s
TRAIN    loss : 1.29986    f1 : 0.37331
save best pt
epoch : 12/40    time : 13s/358s
TRAIN    loss : 1.21970    f1 : 0.40608
save best pt
epo

100%|██████████| 881/881 [00:13<00:00, 65.30it/s]


epoch : 1/40    time : 20s/780s
TRAIN    loss : 4.60596    f1 : 0.13419
save best pt
epoch : 2/40    time : 20s/748s
TRAIN    loss : 2.00649    f1 : 0.20548
save best pt
epoch : 3/40    time : 20s/725s
TRAIN    loss : 1.84696    f1 : 0.18573
epoch : 4/40    time : 20s/711s
TRAIN    loss : 1.80552    f1 : 0.19136
epoch : 5/40    time : 20s/687s
TRAIN    loss : 1.77002    f1 : 0.18406
epoch : 6/40    time : 21s/721s
TRAIN    loss : 1.76175    f1 : 0.21534
save best pt
epoch : 7/40    time : 21s/681s
TRAIN    loss : 1.74805    f1 : 0.19526
epoch : 8/40    time : 20s/634s
TRAIN    loss : 1.72939    f1 : 0.20218
epoch : 9/40    time : 22s/669s
TRAIN    loss : 1.74892    f1 : 0.19670
epoch : 10/40    time : 24s/731s
TRAIN    loss : 1.73476    f1 : 0.19354
epoch : 11/40    time : 21s/598s
TRAIN    loss : 1.72074    f1 : 0.19650
epoch : 12/40    time : 20s/557s
TRAIN    loss : 1.69085    f1 : 0.20553
epoch : 13/40    time : 20s/533s
TRAIN    loss : 1.70393    f1 : 0.19697
epoch : 14/40    time

### 추론

In [118]:
class_pred_df
state_pred=[]
for j in range(15):
    tem_df = class_pred_df[class_pred_df['class']==j]
    tem_png = tem_df["file_name"].tolist()
    tem_path =[]
    for i in tem_png:
        tem_path.append("test\\"+i)
    tem_imgs = [img_load(m) for m in tqdm(tem_path[:])]
    
    tem_dataset = Custom_dataset(np.array(tem_imgs), np.array(["tmp"]*len(tem_imgs)), mode='test',transforms=train_transform)
    tem_loader = DataLoader(tem_dataset, shuffle=False, batch_size=batch_size)
    
    state_models[j].eval()
    tem_f_pred = []
    with torch.no_grad():
        for batch in (tem_loader):
            x = torch.tensor(batch[0], dtype = torch.float32, device = device)
            with torch.cuda.amp.autocast():
                tem_pred = state_models[j](x)
            tem_f_pred.extend(tem_pred.argmax(1).detach().cpu().numpy().tolist())
    state_pred.append(tem_f_pred)


100%|██████████| 52/52 [00:00<00:00, 68.06it/s]


In [119]:
submission = pd.DataFrame(columns = ['file_name', 'class', 'state'])
for j in range(15):
    tmp_df = class_pred_df[class_pred_df['class']==j]
    state_decoder = {val:key for key, val in all_state_unique[j].items()}
    f_result = [state_decoder[result] for result in state_pred[j]]
    tmp_df['state'] = f_result
    submission = pd.concat([submission, tmp_df], axis = 0)


In [120]:
state_decoder = {val:key for key, val in all_state_unique[0].items()}
f_result = [state_decoder[result] for result in state_pred[0]]

In [121]:
submission = submission.sort_index(ascending=True)
submission

,file_name,class,state
0,13000.png,7,good
1,13001.png,6,good
2,13002.png,5,good
3,13003.png,0,good
4,13004.png,9,manipulated_front
...,...,...,...
995,13995.png,14,split_teeth
996,13996.png,13,good
997,13997.png,5,good
998,13998.png,2,faulty_imprint


In [122]:
submission_class_enc = submission['class']

class_decoder = {val:key for key, val in class_unique_enc.items()}
c_result = [class_decoder[result] for result in submission_class_enc]
submission['class'] = c_result

In [123]:
label_pred = submission['class'].str.cat(submission['state'], sep='-')

In [124]:
submission['label']=label_pred

In [125]:
submission

,file_name,class,state,label
0,13000.png,metal_nut,good,metal_nut-good
1,13001.png,leather,good,leather-good
2,13002.png,hazelnut,good,hazelnut-good
3,13003.png,bottle,good,bottle-good
4,13004.png,screw,manipulated_front,screw-manipulated_front
...,...,...,...,...
995,13995.png,zipper,split_teeth,zipper-split_teeth
996,13996.png,wood,good,wood-good
997,13997.png,hazelnut,good,hazelnut-good
998,13998.png,capsule,faulty_imprint,capsule-faulty_imprint


### 제출물 생성

In [126]:
#submission.to_csv("submit0908_36.csv", index = False)

In [127]:
# label_split = submission['label'].str.split("-")
# submission["class"] = label_split.str.get(0)
# submission["state"] = label_split.str.get(1)

# submit = submission[['file_name','class','state','label']]
# submit

In [128]:
#submit.to_csv("submit0906.csv", index = False)

In [129]:
answer = pd.read_excel("답지.xlsx")
# pp =  pd.read_csv("/content/submit.csv")
# count = 0
# for i in range(answer.shape[0]):
#     if answer["class"][i] == submit["class"][i] and answer["state"][i] == submit["state"][i] and answer["label"][i] == submit["label"][i]:
#         count += 1

# acc = count / answer.shape[0] * 100
# print(acc)

In [130]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_labels = train_df["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

submit_labels = submission["label"]
submit_labels = [label_unique[k] for k in submit_labels]


answer_labels = answer["label"]
answer_labels = [label_unique[k] for k in answer_labels]


In [131]:
print(f1_score(answer_labels, submit_labels,average='macro'))

0.4136014640408225
